In [15]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("../dataset/sines.pdf")
docs = loader.load_and_split()

In [16]:
text = docs[0].page_content.replace("\n", " ")

In [14]:
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter

In [17]:
# Chunk text
text_splitter = RecursiveCharacterTextSplitter(chunk_size=600,
                                      chunk_overlap=30)
chunked_documents = text_splitter.split_documents(docs)


In [24]:
chunked_documents[0].page_content.replace("\n", " ")

'Welcome to SINES School of Interdisciplinary Engineering & Sciences is a top-notch research institute, one of its own kinds in the country; parented by National University of Science and Technology Islamabad. It was established in 2007 and is famous for its supercomputing facilities which provide its technical studies with cutting edge resources. SINES NUST’s supercomputer once made it to the list of top 500 supercomputers worldwide. Facilitated by the enabling environment the faculty and students at SINES developed a multi-disciplinary approach to research and education. The result oriented'

In [25]:
chunked_documents[1].page_content.replace("\n", " ")

'The result oriented research by our faculty and students has made significant contributions in the socio-economic growth of the country. The education, training and research activities of SINES have been divided into two main streams, namely: Computational Sciences; and Computational Engineering. At SINES, students are encouraged and facilitated to involve themselves in specialized topics with faculty members who are on the forefront of innovation in their fields and the practitioners who bring real-world, real-time experience to the classroom. The SINES community is driven by a shared'

In [26]:
chunked_documents[2].page_content.replace("\n", " ")

'is driven by a shared purpose: to make a better world through education, research, and innovation. Why SINES? – National University of Sciences and Technology (NUST) World’s Top Universities and Departments NUST is placed at number 400 in the QS World University Ranking 2019. Moreover, in the subject ranking, the university is placed at number 367 in Aeronautical Engineering and at number 273 in Computer Science and Information Systems. SINES improved scores in academic reputation, citations per faculty, faculty/student ratio has helped further consolidate its position this year.'

In [27]:
dataset = chunked_documents[:10]

In [28]:
print(len(dataset))

10


In [36]:
context = []
for doc in dataset:
    page_content = doc.page_content.replace("\n", " ")
    context.append(page_content)
    

In [37]:
context

['Welcome to SINES School of Interdisciplinary Engineering & Sciences is a top-notch research institute, one of its own kinds in the country; parented by National University of Science and Technology Islamabad. It was established in 2007 and is famous for its supercomputing facilities which provide its technical studies with cutting edge resources. SINES NUST’s supercomputer once made it to the list of top 500 supercomputers worldwide. Facilitated by the enabling environment the faculty and students at SINES developed a multi-disciplinary approach to research and education. The result oriented',
 'The result oriented research by our faculty and students has made significant contributions in the socio-economic growth of the country. The education, training and research activities of SINES have been divided into two main streams, namely: Computational Sciences; and Computational Engineering. At SINES, students are encouraged and facilitated to involve themselves in specialized topics wit

In [40]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embeddings = model.encode(sentences)
print(embeddings)

c:\Users\faizan\anaconda3\envs\rag_chatbot\lib\site-packages\bitsandbytes\cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'



c:\Users\faizan\anaconda3\envs\rag_chatbot\lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


[[ 0.02250257 -0.07829178 -0.02303074 ... -0.00827928  0.02652692
  -0.00201897]
 [ 0.04170237  0.00109738 -0.01553418 ... -0.02181629 -0.06359358
  -0.00875285]]


In [42]:
from pinecone import Pinecone
from tqdm.autonotebook import tqdm
pc = Pinecone(api_key="67849698-b32a-42f1-b55d-fb4eb8091d3a")
index = pc.Index("test-index")

In [43]:
batch_size=2 
vector_limit=10000

# questions = question[:vector_limit]
questions = context

import json

for i in tqdm(range(0, len(questions), batch_size)):
    # find end of batch
    i_end = min(i+batch_size, len(questions))
    # create IDs batch
    ids = [str(x) for x in range(i, i_end)]
    # create metadata batch
    metadatas = [{'text': text} for text in questions[i:i_end]]
    # create embeddings
    xc = model.encode(questions[i:i_end])
    # create records list for upsert
    records = zip(ids, xc, metadatas)
    # upsert to Pinecone
    index.upsert(vectors=records)

100%|██████████| 5/5 [00:05<00:00,  1.18s/it]


In [44]:
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0001,
 'namespaces': {'': {'vector_count': 10}},
 'total_vector_count': 10}

In [ ]:
def run_query(query):
  embedding = model.encode(query).tolist()
  results = index.query(top_k=2, vector=embedding, include_metadata=True, include_values=False)
  for result in results['matches']:
    print(f"{round(result['score'], 2)}: {result['metadata']['text']}")